# Blood Pressure KDE Plots

This notebook generates a chart showing KDE plots for systolic and diastolic blood pressure readings. KDE plots provide a "smooth curve" over histogram data.

It requires a copy of "api.ipynb" in the same folder, as this defines functions used to access the web service.

To use it, set the credentials, user details and chart export format in the first code cell before running the notebook.

In [ ]:
# Credentials for the user to log in as
username = ""
password = ""

# Details for the person to report for
firstnames = ""
surname = ""

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
# Base URL for the Health Tracker service
url = "http://localhost:8099"

In [ ]:
%run api.ipynb

In [ ]:
from datetime import date, datetime, timedelta

# Retrieve all-time blood pressure readings
start = date(1900, 1, 1)
end = datetime.now() + timedelta(days=1)

# Log in to the service, get the person ID and retrieve the blood pressure readings
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_daily_blood_pressure_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Plot KDE for systolic and diastolic readings
plt.figure(figsize=(10, 5))
sns.kdeplot(df["systolic"], fill=True, color="red", label="Systolic")
sns.kdeplot(df["diastolic"], fill=True, color="blue", label="Diastolic")

# Set the chart titles and labels
plt.title("KDE Plot of Blood Pressure Readings")
plt.xlabel("Blood Pressure (mmHg)")
plt.ylabel("Density")
plt.legend()
plt.grid(True)

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Export to PNG
export_file_name = "Blood-Pressure-KDE-Chart"
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the chart
plt.show()

# KDE Plots Versus Histograms

The next code cell provides a demonstration of the difference between a KDE plot and histogram of the data


In [ ]:
plt.figure(figsize=(14, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df["systolic"], bins=10, color="red", edgecolor="black", label="Systolic")
sns.histplot(df["diastolic"], bins=10, color="blue", edgecolor="black", label="Diastolic")
plt.title("Histogram of Systolic Readings")
plt.xlabel("Blood Pressure (mmHg)")
plt.ylabel("Count")
plt.legend()

# KDE
plt.subplot(1, 2, 2)
sns.kdeplot(df["systolic"], fill=True, color="red", label="Systolic")
sns.kdeplot(df["diastolic"], fill=True, color="blue", label="Diastolic")
plt.title("KDE Plot of Systolic Readings")
plt.xlabel("Blood Pressure (mmHg)")
plt.ylabel("Density")
plt.legend()

plt.tight_layout()
plt.show()
